In [1]:
import numpy as np
import pandas as pd
import torch
import tensorly as tl
# library for TTD   (Tensorly  (search))

from tensorly.decomposition import parafac
from tensorly.decomposition import tucker
from tensorly.decomposition import tensor_train
from tensorly.decomposition import tensor_train
# different forms of tensor
from sklearn.preprocessing import MinMaxScaler
from tensorly import tt_to_tensor
from tensorly.decomposition import matrix_product_state
# tensor operations


#from math import ceil
#from google.colab import drive
#drive.mount('/content/gdrive')
import torch.nn.init as init
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms, utils
from torch.utils.data import Dataset, DataLoader
import time
import os
import copy
import torch.nn.functional as Fun
print("PyTorch Version:", torch.__version__)
print("Torchvision Version:", torchvision.__version__)
print("GPU is available?", torch.cuda.is_available())

PyTorch Version: 1.13.1
Torchvision Version: 0.14.1
GPU is available? True


In [2]:
dtype = torch.float
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
features = list()
with open("/home/c/cl237/TenBCD/UCI HAR/data/features.txt") as f:
    features = [line.split()[1] for line in f.readlines()]
print('# of Features: {}'.format(len(features)))

# of Features: 561


## get train data

In [4]:
pd_X_train = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/X_train.txt', delim_whitespace=True, header=None)
pd_X_train.columns = features
pd_y_train = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/y_train.txt', delim_whitespace=True, names=['Activity'])

N = len(pd_X_train)

K = 6

x = pd_X_train.values
y = pd_y_train.values

X_train=torch.tensor(x,dtype = torch.float32,device=device)
y_train=torch.tensor(y,dtype = int) #dtype=torch.int,device=device)

y_train = torch.flatten(y_train)

X_train = torch.t(X_train)

y_one_hot = torch.zeros(N, K).scatter_(1, torch.reshape(y_train-1, (N, 1)), 1)
y_one_hot = torch.t(y_one_hot).to(device=device)
y_train = y_train.to(device=device)

## get test data

In [5]:
pd_X_test = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/X_test.txt', delim_whitespace=True, header=None)
pd_X_test.columns = features
pd_y_test = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/y_test.txt', delim_whitespace=True, names=['Activity'])

N_test = len(pd_X_test)

x = pd_X_test.values
y = pd_y_test.values

X_test=torch.tensor(x,dtype = torch.float32, device=device)
y_test=torch.tensor(y,dtype = int)

y_test = torch.flatten(y_test)

X_test = torch.t(X_test)

y_test_one_hot = torch.zeros(N_test, K).scatter_(1, torch.reshape(y_test-1, (N_test, 1)), 1)
y_test_one_hot = torch.t(y_test_one_hot).to(device=device)
y_test = y_test.to(device=device)

In [6]:
def updateV(U1,U2,W,b,rho,gamma): 
    _, d = W.size()
    I = torch.eye(d, device=device) #Returns a 2-D tensor with ones on the diagonal and zeros elsewhere.
    U1 = nn.ReLU()(U1)
    _, col_U2 = U2.size()
    Vstar = torch.mm(torch.inverse(rho*(torch.mm(torch.t(W),W))+gamma*I), rho*torch.mm(torch.t(W),U2-b.repeat(1,col_U2))+gamma*U1)
    # check how to get V* W*
    # torch.mm*=(input,mat2) performs a matrix multiplication of the matrices input and mat2.
    #   if input is a (n*m) tensor,mat2 is a (m*p) tensor, output will be (n*p) tensor
    return Vstar

def updateWb_org(U, V, W, b, alpha, rho): 
    d,N = V.size()
    I = torch.eye(d, device=device)
    _, col_U = U.size()
    Wstar = torch.mm(alpha*W+rho*torch.mm(U-b.repeat(1,col_U),torch.t(V)),torch.inverse(alpha*I+rho*(torch.mm(V,torch.t(V)))))
    bstar = 0*(alpha*b+rho*torch.sum(U-torch.mm(Wstar,V), dim=1).reshape(b.size()))/(rho*N+alpha)
    return Wstar, bstar

def updateWb(U, V, W, b, W_tensor_rec, alpha, rho,tau): 
    W_tensor_rec = torch.as_tensor(W_tensor_rec,device=device).float()
    W_tensor2matrix = W_tensor_rec.reshape(W.shape)
    d,N = V.size()
    I = torch.eye(d, device=device)
    _, col_U = U.size()
    Wstar = torch.mm(alpha*W+tau*W_tensor2matrix+rho*torch.mm(U-b.repeat(1,col_U),torch.t(V)),torch.inverse((alpha+tau)*I+rho*(torch.mm(V,torch.t(V)))))
    bstar = 0*(alpha*b+rho*torch.sum(U-torch.mm(Wstar,V), dim=1).reshape(b.size()))/(rho*N+alpha)
    return Wstar, bstar

# U update  E(1) in global convergent
def relu_prox(a, b, gamma, d, N):
    val = torch.empty(d,N, device=device)
    x = (a+gamma*b)/(1+gamma)
    y = torch.min(b,torch.zeros(d,N, device=device))

    val = torch.where(a+gamma*b < 0, y, torch.zeros(d,N, device=device))
    val = torch.where(((a+gamma*b >= 0) & (b >=0)) | ((a*(gamma-np.sqrt(gamma*(gamma+1))) <= gamma*b) & (b < 0)), x, val)
    val = torch.where((-a <= gamma*b) & (gamma*b <= a*(gamma-np.sqrt(gamma*(gamma+1)))), b, val)
    return val

In [7]:
########### parameter setup
#df = pd.DataFrame()
#df.to_csv('C:/Users/Mark/Desktop/HAR_5layers_rank40_100times.csv')
niter = 500
rank = 80
tau = 3
gamma = 0.5
rho = 0.5
alpha = 1



print ("rank=",rank, "tau=",tau, "gamma=",gamma, "rho=",rho, "alpha",alpha)

loss1 = np.empty(niter)
loss2 = np.empty(niter)
accuracy_train = np.empty(niter)
accuracy_test = np.empty(niter)
time1 = np.empty(niter)

results = torch.zeros(1, 5, niter)



for Out_iter in range(1):
    rank_initial = 400
    seed = 10 + 10*Out_iter
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.manual_seed(seed)
    d0 = 561 #561 =3*11*17

    d1 =  1024    # 2^10
    d2 =  1024  
    d3 = 1024
    d4 = 512      # 2^9
    d5 = 512
    d6 = 6 


    W1 = 0.2*init.xavier_uniform_(torch.empty(d1, d0, device=device), gain=1.0)
    W1_torch_tensor = W1.reshape((6,22,34,2,2,2,2,2,2,2))
    W1_tl_tensor = tl.tensor(W1_torch_tensor.cpu().numpy())
    factors1 = tensor_train(W1_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W1_tl_tensor_rec = tt_to_tensor(factors1)
    b1 = 0*torch.ones(d1, 1, device=device) # 0 is stable


    W2 = 0.2*init.xavier_uniform_(torch.empty(d2, d1, device=device), gain=1.0)
    W2_torch_tensor = W2.reshape((4,4,4,4,4,4,4,4,4,4)) # 8 number of 4s, 2 number of 8s
    W2_tl_tensor = tl.tensor(W2_torch_tensor.cpu().numpy())
    factors2 = tensor_train(W2_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    # we have 9 rank_initial (r1,...,r9) here plus 2 of 1s (r0 & r10).
    # factors: set of tensor cores
    # http://tensorly.org/stable/user_guide/tensor_decomposition.html
    # http://tensorly.org/stable/modules/generated/tensorly.decomposition.tensor_train.html#tensorly.decomposition.tensor_train
    W2_tl_tensor_rec = tt_to_tensor(factors2)
    # tt_to_tensor(factors):Re-assembles ‘factors’, which represent a tensor in TT format into the corresponding full tensor
    #      facros: list of 3d-arrays tt-cores           output_tensor: ndarray      
    b2 = 0*torch.ones(d2, 1, device=device)


    W3 = 0.2*init.xavier_uniform_(torch.empty(d3, d2, device=device), gain=1.0)
    W3_torch_tensor = W3.reshape((4,4,4,4,4,4,4,4,4,4)) # 8 number of 4s, 2 number of 8s
    W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())
    factors3 = tensor_train(W3_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W3_tl_tensor_rec = tt_to_tensor(factors3)
    b3 = 0*torch.ones(d3, 1, device=device)

    W4 = 0.2*init.xavier_uniform_(torch.empty(d4, d3, device=device), gain=1.0)
    W4_torch_tensor = W4.reshape((4,4,4,4,4,4,4,4,4,2)) # 8 number of 4s, 2 number of 8s
    W4_tl_tensor = tl.tensor(W4_torch_tensor.cpu().numpy())
    factors4 = tensor_train(W4_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W4_tl_tensor_rec = tt_to_tensor(factors4)
    b4 = 0*torch.ones(d4, 1, device=device)

    W5 = 0.2*init.xavier_uniform_(torch.empty(d5, d4, device=device), gain=1.0)
    W5_torch_tensor = W5.reshape((4,4,4,4,4,4,4,4,4)) # 8 number of 4s, 2 number of 8s
    W5_tl_tensor = tl.tensor(W5_torch_tensor.cpu().numpy())
    factors5 = tensor_train(W5_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W5_tl_tensor_rec = tt_to_tensor(factors5)
    b5 = 0*torch.ones(d5, 1, device=device)


    W6 = 0.2*init.xavier_uniform_(torch.empty(d6, d5, device=device), gain=1.0)
    b6 = 0*torch.ones(d6, 1, device=device)

    # W4 = 0.01*torch.randn(d4, d3, device=device)
    # W4_torch_tensor = W4.reshape((50,30,K))
    # W4_tl_tensor = tl.tensor(W4_torch_tensor.cpu().numpy())
    # core4, tucker_factors4 = tucker(W4_tl_tensor, rank=(40,28,K), init='svd', tol=10e-5, random_state=12345)
    # W4_tl_tensor_rec = tl.tucker_to_tensor((core4, tucker_factors4))
    # b4 = 0.1*torch.ones(d4, 1, device=device)

    U1 = torch.addmm(b1.repeat(1, N), W1, X_train)
    V1 = nn.ReLU()(U1)
    U2 = torch.addmm(b2.repeat(1, N), W2, V1)
    V2 = nn.ReLU()(U2)
    U3 = torch.addmm(b3.repeat(1, N), W3, V2)
    V3 = nn.ReLU()(U3)
    U4 = torch.addmm(b4.repeat(1, N), W4, V3)
    V4 = nn.ReLU()(U4)
    U5 = torch.addmm(b5.repeat(1, N), W5, V4)
    V5 = nn.ReLU()(U5)
    U6 = torch.addmm(b6.repeat(1, N), W6, V5)
    V6 = U6 
    # U4 = torch.addmm(b4.repeat(1, N), W4, V3)
    # V4 = U4



    # Iterations
    print('Train on', N, 'samples, validate on', N_test, 'samples')
    for k in range(niter):
        start = time.time()

 # update for last layer
        # update V4
        V6 = (y_one_hot + gamma*U6 + alpha*V6)/(1 + gamma + alpha)

        # update U4 
        U6 = (gamma*V6 + rho*(torch.mm(W6,V5) + b6.repeat(1,N)))/(gamma + rho)

        # update W4 and b4
        W6, b6 = updateWb_org(U6,V5,W6,b6,alpha,rho)
        # W3_torch_tensor = W3.reshape((d3,4,4,4,4,4,4))
        # W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())
        # factors3 = tensor_train(W3_tl_tensor, (1,rank,rank,rank,rank,rank,rank,1))
        # W3_tl_tensor_rec = tt_to_tensor(factors3)


        # update for 5th layer
        # update V3
        V5 = updateV(U5,U6,W6,b6,rho,gamma)

        # update U3
        U5 = relu_prox(V5,(rho*torch.addmm(b5.repeat(1,N), W5, V4) + alpha*U5)/(rho + alpha),(rho + alpha)/gamma,d5,N)

        # update W3 and b3
        W5, b5 = updateWb(U5,V4,W5,b5,W5_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)   
        W5_torch_tensor = W5.reshape((4,4,4,4,4,4,4,4,4))
        W5_tl_tensor = tl.tensor(W5_torch_tensor.cpu().numpy())  # transfer tensorly package
        factors5 = tensor_train(W5_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,1))
        #set of tensor cores
        W5_tl_tensor_rec = tt_to_tensor(factors5)


  # update for 4th layer
        # update V3
        V4 = updateV(U4,U5,W5,b5,rho,gamma)

        # update U3
        U4 = relu_prox(V4,(rho*torch.addmm(b4.repeat(1,N), W4, V3) + alpha*U4)/(rho + alpha),(rho + alpha)/gamma,d4,N)

        # update W3 and b3
        W4, b4 = updateWb(U4,V3,W4,b4,W4_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)   
        W4_torch_tensor = W4.reshape((4,4,4,4,4,4,4,4,4,2))
        W4_tl_tensor = tl.tensor(W4_torch_tensor.cpu().numpy())  # transfer tensorly package
        factors4 = tensor_train(W4_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        #set of tensor cores
        W4_tl_tensor_rec = tt_to_tensor(factors4)

  # update for 3nd layer
        # update V3
        V3 = updateV(U3,U4,W4,b4,rho,gamma)

        # update U3
        U3 = relu_prox(V3,(rho*torch.addmm(b3.repeat(1,N), W3, V2) + alpha*U3)/(rho + alpha),(rho + alpha)/gamma,d3,N)

        # update W3 and b3
        W3, b3 = updateWb(U3,V2,W3,b3,W3_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)   
        W3_torch_tensor = W3.reshape((4,4,4,4,4,4,4,4,4,4))
        W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())  # transfer tensorly package
        factors3 = tensor_train(W3_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        #set of tensor cores
        W3_tl_tensor_rec = tt_to_tensor(factors3)

 # update for 2nd layer
        # update V2
        V2 = updateV(U2,U3,W3,b3,rho,gamma)

        # update U2
        U2 = relu_prox(V2,(rho*torch.addmm(b2.repeat(1,N), W2, V1) + alpha*U2)/(rho + alpha),(rho + alpha)/gamma,d2,N)

        # update W2 and b2
        W2, b2 = updateWb(U2,V1,W2,b2,W2_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)   
        W2_torch_tensor = W2.reshape((4,4,4,4,4,4,4,4,4,4))
        W2_tl_tensor = tl.tensor(W2_torch_tensor.cpu().numpy()) 
        factors2 = tensor_train(W2_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        W2_tl_tensor_rec = tt_to_tensor(factors2)


# update for 1st layer
        # update V1
        V1 = updateV(U1,U2,W2,b2,rho,gamma)

        # update U1
        U1 = relu_prox(V1,(rho*torch.addmm(b1.repeat(1,N), W1, X_train) + alpha*U1)/(rho + alpha),(rho + alpha)/gamma,d1,N)

        # update W1 and b1
        W1, b1 = updateWb(U1,X_train,W1,b1,W1_tl_tensor_rec, alpha,rho,tau)

        # G update
        W1_torch_tensor = W1.reshape((6,22,34,2,2,2,2,2,2,2))
        W1_tl_tensor = tl.tensor(W1_torch_tensor.cpu().numpy())
        factors1 = tensor_train(W1_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        W1_tl_tensor_rec = tt_to_tensor(factors1)




 # prediction for trainning data
        a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), X_train))
        #a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), W1, x_train))
        a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_train))
        a3_train = nn.ReLU()(torch.addmm(b3.repeat(1, N), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_train))
        a4_train = nn.ReLU()(torch.addmm(b4.repeat(1, N), torch.as_tensor(W4_tl_tensor_rec,device=device).reshape((d4, d3)).float(), a3_train))
        a5_train = nn.ReLU()(torch.addmm(b5.repeat(1, N), torch.as_tensor(W5_tl_tensor_rec,device=device).reshape((d5, d4)).float(), a4_train))
        #print(torch.addmm(b4.repeat(1, N), W4, a3_train))
        pred = torch.argmax(torch.addmm(b6.repeat(1, N), W6, a5_train), dim=0)
        # check argmax and addmm, dim=0

 #Prediction for test data
        a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), X_test))
        #a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), W1, x_test))
        a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_test))
        a3_test = nn.ReLU()(torch.addmm(b3.repeat(1, N_test), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_test))
        a4_test = nn.ReLU()(torch.addmm(b4.repeat(1, N_test), torch.as_tensor(W4_tl_tensor_rec,device=device).reshape((d4, d3)).float(), a3_test))
        a5_test = nn.ReLU()(torch.addmm(b5.repeat(1, N_test), torch.as_tensor(W5_tl_tensor_rec,device=device).reshape((d5, d4)).float(), a4_test))
        pred_test = torch.argmax(torch.addmm(b6.repeat(1, N_test), W6, a5_test), dim=0)

        # a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), x_train))
        # a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_train))
        # # a3_train = nn.ReLU()(torch.addmm(b3.repeat(1, N), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)), a2_train))
        # #print(torch.addmm(b4.repeat(1, N), W4, a3_train))
        # pred = torch.argmax(torch.addmm(b3.repeat(1, N), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_train), dim=0)

        # a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), x_test))
        # a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_test))
        # # a3_test = nn.ReLU()(torch.addmm(b3.repeat(1, N_test), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)), a2_test))
        # pred_test = torch.argmax(torch.addmm(b3.repeat(1, N_test), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_test), dim=0)

        # a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), W1, x_train))
        # a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N), W2, a1_train))

        # #print(torch.addmm(b4.repeat(1, N), W4, a3_train))
        # pred = torch.argmax(torch.addmm(b3.repeat(1, N), W3, a2_train), dim=0)

        # a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), W1, x_test))
        # a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test), W2, a1_test))
        # pred_test = torch.argmax(torch.addmm(b3.repeat(1, N_test), W3, a2_test), dim=0)




   #emperical loss
        loss1[k] = gamma/2*torch.pow(torch.dist(V6,y_one_hot,2),2).cpu().numpy()
        # torch.pow:Takes the power of each element in input with exponent and returns a tensor with the result.
        # torch.disk: Returns the p-norm of (input - other)
        # Eq (5) in paper
        loss2[k] = loss1[k] + rho/2*torch.pow(torch.dist(torch.addmm(b1.repeat(1,N), W1, X_train),U1,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b2.repeat(1,N), W2, V1),U2,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b3.repeat(1,N), W3, V2),U3,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b4.repeat(1,N), W4, V3),U4,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b5.repeat(1,N), W5, V4),U5,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b6.repeat(1,N), W6, V5),U6,2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V1,nn.ReLU()(U1),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V2,nn.ReLU()(U2),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V3,nn.ReLU()(U3),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V4,nn.ReLU()(U4),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V5,nn.ReLU()(U5),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V6,U6,2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W1.reshape((6,22,34,2,2,2,2,2,2,2)),torch.as_tensor(W1_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W2.reshape((4,4,4,4,4,4,4,4,4,4)),torch.as_tensor(W2_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W3.reshape((4,4,4,4,4,4,4,4,4,4)),torch.as_tensor(W3_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W4.reshape((4,4,4,4,4,4,4,4,4,2)),torch.as_tensor(W4_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W5.reshape((4,4,4,4,4,4,4,4,4)),torch.as_tensor(W5_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \

        # +tau/2*torch.pow(torch.dist(W3.reshape((d3,4,4,4,4,4,4)),torch.as_tensor(W3_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() 
        # +tau/2*torch.pow(torch.dist(W4.reshape((50,30,K)),torch.as_tensor(W4_tl_tensor_rec,device=device),2),2).cpu().numpy() 

        # compute training accuracy
        correct_train = pred == y_train-1
        accuracy_train[k] = np.mean(correct_train.cpu().numpy())

        # compute validation accuracy
        correct_test = pred_test == y_test-1
        accuracy_test[k] = np.mean(correct_test.cpu().numpy())

        # compute training time
        stop = time.time()
        duration = stop - start
        time1[k] = duration

        # print results
        print('Repeatition', Out_iter + 1, 'Epoch', k + 1, '/', niter, '\n', 
              '-', 'time:', time1[k], '-', 'sq_loss:', loss1[k], '-', 'tot_loss:', loss2[k], 
              '-', 'acc:', accuracy_train[k], '-', 'val_acc:', accuracy_test[k])

##############
############## compute CR
    factors1_shape=[f.shape for f in factors1]
    Sum_of_variables_factors1=sum(list(x*y*z for x,y,z in factors1_shape))
    factors2_shape=[f.shape for f in factors2]
    Sum_of_variables_factors2=sum(list(x*y*z for x,y,z in factors2_shape))
    factors3_shape=[f.shape for f in factors3]
    Sum_of_variables_factors3=sum(list(x*y*z for x,y,z in factors3_shape))
    factors4_shape=[f.shape for f in factors4]
    Sum_of_variables_factors4=sum(list(x*y*z for x,y,z in factors4_shape))
    factors5_shape=[f.shape for f in factors5]
    Sum_of_variables_factors5=sum(list(x*y*z for x,y,z in factors5_shape))

    total_variabels=Sum_of_variables_factors1+Sum_of_variables_factors2+Sum_of_variables_factors3+Sum_of_variables_factors4+Sum_of_variables_factors5

    CR_1=((total_variabels)+(d5*d6))/(d0*d1+d1*d2+d2*d3+d3*d4+d4*d5+d5*d6)
    CR_2=(total_variabels)/(d0*d1+d1*d2+d2*d3+d3*d4+d4*d5)
    print("CR_1 = "+str(CR_1)+"   "+"CR_2 = " +str(CR_2))


    results[Out_iter,0,:] = torch.tensor(loss1)
    results[Out_iter,1,:] = torch.tensor(loss2)
    results[Out_iter,2,:] = torch.tensor(accuracy_train)
    results[Out_iter,3,:] = torch.tensor(accuracy_test)
    results[Out_iter,4,:] = torch.tensor(time1)
    CR=(CR_1,CR_2)

#     #this postion to add new row into existing table
#         df=pd.read_csv('C:/Users/Mark/Desktop/HAR_5layers_rank40_100times.csv')
#         new_row = {'rank':rank, 'CR_1':CR_1, 'CR_2':CR_2, 'tau':tau, 'gamma':gamma, 'rho':rho, 'alpha':alpha, \
#                    'loss1':loss1[niter-1], 'loss2':loss2[niter-1], 'accuracy_train':accuracy_train[niter-1], \
#                    'accuracy_test':accuracy_test[niter-1], 'time':time1[niter-1],'seed':seed} 
#         df=df.append(new_row,ignore_index=True)
#         df.to_csv('C:/Users/Mark/Desktop/HAR_5layers_rank40_100times.csv',index=False)


filename= "XavierUniform_" + "niter_"+ str(niter) + "rank_" + str(rank) + "tau_" + str(tau) + "gamma_" + str(gamma) + \
"rho_" + str(rho) + "alpha_" + str(alpha) + ".mat"
from scipy.io import savemat
%cd /home/c/cl237/TenBCD/UCI HAR/5 hidden layers/Different Initialization/
savemat (filename, {'results': torch.Tensor.numpy(results), 'Compression Ratio':CR})
#this position to save table into matlab
#df.to_csv('C:/Users/Mark/Desktop/result_compress_123_layer_highrank.csv')
##Changing Folder
 #%cd '/content/gdrive/MyDrive/Colab/Tensor-BCD-for-DNN/Experiments/DataSaved'
 #savemat("ThreeLayer_rank160.mat", {'results': torch.Tensor.numpy(results)})    

rank= 80 tau= 3 gamma= 0.5 rho= 0.5 alpha 1
Train on 7352 samples, validate on 2947 samples
Repeatition 1 Epoch 1 / 500 
 - time: 3.3170061111450195 - sq_loss: 661.6710205078125 - tot_loss: 962.7074280442848 - acc: 0.17478237214363437 - val_acc: 0.167288768238887
Repeatition 1 Epoch 2 / 500 
 - time: 2.860927104949951 - sq_loss: 294.07598876953125 - tot_loss: 484.4423112773802 - acc: 0.18620783460282916 - val_acc: 0.17645062775704107
Repeatition 1 Epoch 3 / 500 
 - time: 2.822197914123535 - sq_loss: 163.58180236816406 - tot_loss: 263.883510355372 - acc: 0.1749183895538629 - val_acc: 0.166610111978283
Repeatition 1 Epoch 4 / 500 
 - time: 2.8540472984313965 - sq_loss: 89.09494018554688 - tot_loss: 148.20156379742548 - acc: 0.1749183895538629 - val_acc: 0.166610111978283
Repeatition 1 Epoch 5 / 500 
 - time: 2.8872599601745605 - sq_loss: 48.01134490966797 - tot_loss: 85.97253378527239 - acc: 0.1749183895538629 - val_acc: 0.166610111978283
Repeatition 1 Epoch 6 / 500 
 - time: 2.854246139

Repeatition 1 Epoch 48 / 500 
 - time: 2.7758679389953613 - sq_loss: 0.0004084503452759236 - tot_loss: 1.2991925797941803 - acc: 0.5478781284004353 - val_acc: 0.5310485239226331
Repeatition 1 Epoch 49 / 500 
 - time: 2.8293449878692627 - sq_loss: 0.000381750549422577 - tot_loss: 1.2592062403091404 - acc: 0.5485582154515778 - val_acc: 0.5310485239226331
Repeatition 1 Epoch 50 / 500 
 - time: 2.7669570446014404 - sq_loss: 0.0003570599656086415 - tot_loss: 1.2211416346071928 - acc: 0.5525027203482046 - val_acc: 0.5310485239226331
Repeatition 1 Epoch 51 / 500 
 - time: 2.865910291671753 - sq_loss: 0.00033427696325816214 - tot_loss: 1.1769297117189126 - acc: 0.5575353645266594 - val_acc: 0.5313878520529352
Repeatition 1 Epoch 52 / 500 
 - time: 2.792396306991577 - sq_loss: 0.0003134086146019399 - tot_loss: 1.1377173734344979 - acc: 0.5639281828073993 - val_acc: 0.5330844927044452
Repeatition 1 Epoch 53 / 500 
 - time: 2.8082211017608643 - sq_loss: 0.0002941115526482463 - tot_loss: 1.0868324

Repeatition 1 Epoch 95 / 500 
 - time: 2.7742269039154053 - sq_loss: 2.865177702915389e-05 - tot_loss: 0.3522958973792356 - acc: 0.8065832426550599 - val_acc: 0.7723108245673567
Repeatition 1 Epoch 96 / 500 
 - time: 2.8498504161834717 - sq_loss: 2.7211714041186497e-05 - tot_loss: 0.3435066642794027 - acc: 0.80930359085963 - val_acc: 0.7743467933491687
Repeatition 1 Epoch 97 / 500 
 - time: 2.7989277839660645 - sq_loss: 2.5860144887701608e-05 - tot_loss: 0.3526974258554674 - acc: 0.8128400435255713 - val_acc: 0.7784187309127927
Repeatition 1 Epoch 98 / 500 
 - time: 2.784961700439453 - sq_loss: 2.457087794027757e-05 - tot_loss: 0.33941718610753924 - acc: 0.8148803046789989 - val_acc: 0.7807940278249067
Repeatition 1 Epoch 99 / 500 
 - time: 2.783499002456665 - sq_loss: 2.336964098503813e-05 - tot_loss: 0.3330163651211251 - acc: 0.8176006528835691 - val_acc: 0.7831693247370207
Repeatition 1 Epoch 100 / 500 
 - time: 2.8244359493255615 - sq_loss: 2.2211879695532843e-05 - tot_loss: 0.3317

Repeatition 1 Epoch 141 / 500 
 - time: 2.7693183422088623 - sq_loss: 3.6993537833041046e-06 - tot_loss: 0.1901927966928696 - acc: 0.9005712731229597 - val_acc: 0.8710553104852392
Repeatition 1 Epoch 142 / 500 
 - time: 2.797130584716797 - sq_loss: 3.5830450997309526e-06 - tot_loss: 0.17425993827251318 - acc: 0.9009793253536452 - val_acc: 0.8713946386155412
Repeatition 1 Epoch 143 / 500 
 - time: 2.7576775550842285 - sq_loss: 3.464802148300805e-06 - tot_loss: 0.17066809119309312 - acc: 0.9012513601741022 - val_acc: 0.8720732948761453
Repeatition 1 Epoch 144 / 500 
 - time: 2.7778985500335693 - sq_loss: 3.355077069500112e-06 - tot_loss: 0.17137175681266115 - acc: 0.9016594124047879 - val_acc: 0.8751272480488632
Repeatition 1 Epoch 145 / 500 
 - time: 2.7867214679718018 - sq_loss: 3.2466728043800686e-06 - tot_loss: 0.16528850192126754 - acc: 0.9015233949945594 - val_acc: 0.8764845605700713
Repeatition 1 Epoch 146 / 500 
 - time: 2.7968251705169678 - sq_loss: 3.1433348794962512e-06 - tot_

Repeatition 1 Epoch 187 / 500 
 - time: 2.812690019607544 - sq_loss: 1.336239506599668e-06 - tot_loss: 0.13371201769212604 - acc: 0.926822633297062 - val_acc: 0.9087207329487614
Repeatition 1 Epoch 188 / 500 
 - time: 2.835651397705078 - sq_loss: 1.3205475397626287e-06 - tot_loss: 0.15559485296033948 - acc: 0.9269586507072906 - val_acc: 0.9090600610790635
Repeatition 1 Epoch 189 / 500 
 - time: 2.7866499423980713 - sq_loss: 1.3087940260447795e-06 - tot_loss: 0.1415278413978278 - acc: 0.9270946681175191 - val_acc: 0.9093993892093655
Repeatition 1 Epoch 190 / 500 
 - time: 2.8695342540740967 - sq_loss: 1.2964392226422206e-06 - tot_loss: 0.1405101303931442 - acc: 0.9272306855277476 - val_acc: 0.9093993892093655
Repeatition 1 Epoch 191 / 500 
 - time: 2.846831798553467 - sq_loss: 1.2859710523116519e-06 - tot_loss: 0.14415693000750984 - acc: 0.9276387377584331 - val_acc: 0.9097387173396675
Repeatition 1 Epoch 192 / 500 
 - time: 2.7894787788391113 - sq_loss: 1.273480620511691e-06 - tot_loss

...

Repeatition 1 Epoch 279 / 500 
 - time: 2.805680274963379 - sq_loss: 8.155911928042769e-07 - tot_loss: 0.10924193533474735 - acc: 0.9596028291621328 - val_acc: 0.9389209365456397
Repeatition 1 Epoch 280 / 500 
 - time: 2.8917500972747803 - sq_loss: 8.132436732921633e-07 - tot_loss: 0.12056348566927522 - acc: 0.9597388465723613 - val_acc: 0.9392602646759416
Repeatition 1 Epoch 281 / 500 
 - time: 2.832566499710083 - sq_loss: 8.102566653178656e-07 - tot_loss: 0.10817261912438036 - acc: 0.9597388465723613 - val_acc: 0.9395995928062436
Repeatition 1 Epoch 282 / 500 
 - time: 2.8460042476654053 - sq_loss: 8.08729964774102e-07 - tot_loss: 0.11928061595214201 - acc: 0.9601468988030468 - val_acc: 0.9392602646759416
Repeatition 1 Epoch 283 / 500 
 - time: 2.8179383277893066 - sq_loss: 8.066113537097408e-07 - tot_loss: 0.12411754956787435 - acc: 0.9602829162132753 - val_acc: 0.9392602646759416
Repeatition 1 Epoch 284 / 500 
 - time: 2.8069307804107666 - sq_loss: 8.063984182626882e-07 - tot_loss:

Repeatition 1 Epoch 325 / 500 
 - time: 2.750732421875 - sq_loss: 7.289005452548736e-07 - tot_loss: 0.12231543044881876 - acc: 0.9680359085963003 - val_acc: 0.9443501866304717
Repeatition 1 Epoch 326 / 500 
 - time: 2.6803057193756104 - sq_loss: 7.282431511157483e-07 - tot_loss: 0.11951793974591962 - acc: 0.9683079434167573 - val_acc: 0.9446895147607737
Repeatition 1 Epoch 327 / 500 
 - time: 2.6940178871154785 - sq_loss: 7.261968448801781e-07 - tot_loss: 0.12082975297581844 - acc: 0.9685799782372143 - val_acc: 0.9446895147607737
Repeatition 1 Epoch 328 / 500 
 - time: 2.687427282333374 - sq_loss: 7.24057770185027e-07 - tot_loss: 0.10940366800652557 - acc: 0.9687159956474428 - val_acc: 0.9446895147607737
Repeatition 1 Epoch 329 / 500 
 - time: 2.68622088432312 - sq_loss: 7.209629870885692e-07 - tot_loss: 0.12391020962467669 - acc: 0.9689880304678999 - val_acc: 0.9450288428910757
Repeatition 1 Epoch 330 / 500 
 - time: 2.7242860794067383 - sq_loss: 7.186031893979816e-07 - tot_loss: 0.11

Repeatition 1 Epoch 371 / 500 
 - time: 2.770777940750122 - sq_loss: 6.592767363144958e-07 - tot_loss: 0.124574576224594 - acc: 0.9741566920565833 - val_acc: 0.9504580929759077
Repeatition 1 Epoch 372 / 500 
 - time: 2.8037610054016113 - sq_loss: 6.585512437595753e-07 - tot_loss: 0.09958214698509793 - acc: 0.9740206746463548 - val_acc: 0.9507974211062097
Repeatition 1 Epoch 373 / 500 
 - time: 2.7385292053222656 - sq_loss: 6.579221007996239e-07 - tot_loss: 0.09980788071412716 - acc: 0.9740206746463548 - val_acc: 0.9511367492365117
Repeatition 1 Epoch 374 / 500 
 - time: 2.7800955772399902 - sq_loss: 6.581055345122877e-07 - tot_loss: 0.1390850302149398 - acc: 0.9742927094668118 - val_acc: 0.9511367492365117
Repeatition 1 Epoch 375 / 500 
 - time: 2.7661707401275635 - sq_loss: 6.580598892469425e-07 - tot_loss: 0.11480962466386169 - acc: 0.9742927094668118 - val_acc: 0.9514760773668137
Repeatition 1 Epoch 376 / 500 
 - time: 2.7890310287475586 - sq_loss: 6.568189405697922e-07 - tot_loss: 

Repeatition 1 Epoch 417 / 500 
 - time: 2.745434522628784 - sq_loss: 6.138815251688357e-07 - tot_loss: 0.11719528860838957 - acc: 0.9776931447225244 - val_acc: 0.9531727180183237
Repeatition 1 Epoch 418 / 500 
 - time: 2.845343589782715 - sq_loss: 6.117272164374299e-07 - tot_loss: 0.11297933249313585 - acc: 0.977829162132753 - val_acc: 0.9535120461486257
Repeatition 1 Epoch 419 / 500 
 - time: 2.8812317848205566 - sq_loss: 6.111657739893417e-07 - tot_loss: 0.120605357410567 - acc: 0.977829162132753 - val_acc: 0.9531727180183237
Repeatition 1 Epoch 420 / 500 
 - time: 2.7555992603302 - sq_loss: 6.10428912750649e-07 - tot_loss: 0.12184467655419273 - acc: 0.97810119695321 - val_acc: 0.9528333898880217
Repeatition 1 Epoch 421 / 500 
 - time: 2.8354310989379883 - sq_loss: 6.097562845752691e-07 - tot_loss: 0.1085394648897493 - acc: 0.977829162132753 - val_acc: 0.9531727180183237
Repeatition 1 Epoch 422 / 500 
 - time: 2.7851226329803467 - sq_loss: 6.083042194404698e-07 - tot_loss: 0.12073288

Repeatition 1 Epoch 463 / 500 
 - time: 1.4344096183776855 - sq_loss: 5.716491955354286e-07 - tot_loss: 0.12585430141742238 - acc: 0.9808215451577802 - val_acc: 0.9535120461486257
Repeatition 1 Epoch 464 / 500 
 - time: 1.4283721446990967 - sq_loss: 5.708740218324238e-07 - tot_loss: 0.10547925080471865 - acc: 0.9808215451577802 - val_acc: 0.9535120461486257
Repeatition 1 Epoch 465 / 500 
 - time: 1.4205472469329834 - sq_loss: 5.693449907084869e-07 - tot_loss: 0.10618927011937651 - acc: 0.9809575625680087 - val_acc: 0.9535120461486257
Repeatition 1 Epoch 466 / 500 
 - time: 1.412541389465332 - sq_loss: 5.685473070116132e-07 - tot_loss: 0.11066049247950482 - acc: 0.9809575625680087 - val_acc: 0.9535120461486257
Repeatition 1 Epoch 467 / 500 
 - time: 1.4237964153289795 - sq_loss: 5.682207984136767e-07 - tot_loss: 0.12440476191051097 - acc: 0.9808215451577802 - val_acc: 0.9535120461486257
Repeatition 1 Epoch 468 / 500 
 - time: 1.4056994915008545 - sq_loss: 5.675791499015759e-07 - tot_los

In [8]:
seed

10